# Exploratory Data Analysis (EDA) on Steam Games Dataset

In this project, I've compiled and analyzed a dataset with **27,075 games** from **Steam**, the popular digital distribution platform for PC games. This dataset is updated from the platform's inception until April 2019.

The goal of this analysis is to answer 10 key questions about the trends and performance of games on Steam, focusing on aspects like popularity, genre trends, developer success, and user reviews. These questions include:

1. What are the top 10 most-played games?
2. Which is the best-selling video game genre?
3. Which genres have the highest average reviews?
4. How much has the top-selling developer earned?
5. How do Early Access games affect review scores?
6. Which developer(s) have released the most diverse and successful games?
7. Which month saw the highest number of game releases?
8. How many hours are played per genre on average each month?
9. Do Free-to-Play games have better or worse reviews?
10. How has the number of achievements per game evolved over time?

Feel free to dive into the analysis and explore these insights!



# Importing Libraries

In [185]:
import pandas as pd
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import plotly.graph_objects as go
import os

import urllib.request
from zipfile import ZipFile

# Loading the data

In [186]:
# Create folder if doesn't exist
if not os.path.exists('../archive'):
    os.makedirs('../archive')
    
if not os.path.exists('../data'):
    os.makedirs('../data')

# Download and unzip the dataset
url = 'https://neo-mastermind.s3.amazonaws.com/uploads/froala_editor/files/SteamDataset-220424-201858.zip'
urllib.request.urlretrieve(url, '../archive/steam_dataset.zip')

# Extract the zip 
with ZipFile('../archive/steam_dataset.zip') as steam_dataset:
    steam_dataset.extractall('../data/')
    steam_dataset.filelist
    df = pd.read_csv(steam_dataset.open('steam.csv'), index_col=[0])
    
print(os.listdir('../data/'))

['steam.csv', 'steam_games_cleaned.csv']


In [187]:
df.head()

,name,release_date,developer,achievements,positive_ratings,negative_ratings,avg_hours_per_user,price,sells,action,adventure,rpg,simulation,strategy,racing,free_to_play,early_access
0,Counter-Strike,2000-11-01,Valve,0,124534,3339,17612,7.19€,15000000.0,True,False,False,False,False,False,False,False
1,Team Fortress Classic,1999-04-01,Valve,0,3318,633,277,3.99€,7500000.0,True,False,False,False,False,False,False,False
2,Day of Defeat,2003-05-01,Valve,0,3416,398,187,3.99€,7500000.0,True,False,False,False,False,False,False,False
3,Deathmatch Classic,2001-06-01,Valve,0,1273,267,258,3.99€,7500000.0,True,False,False,False,False,False,False,False
4,Half-Life: Opposing Force,1999-11-01,Gearbox Software,0,5250,288,624,3.99€,7500000.0,True,False,False,False,False,False,False,False


In [188]:
df.columns

Index(['name', 'release_date', 'developer', 'achievements', 'positive_ratings',
       'negative_ratings', 'avg_hours_per_user', 'price', 'sells', 'action',
       'adventure', 'rpg', 'simulation', 'strategy', 'racing', 'free_to_play',
       'early_access'],
      dtype='object')

# Data Cleaning and Manipulation

## Identify Duplicate or Irrelevant Data

In [189]:
df[df[['name', 'developer']].duplicated()].head()

,name,release_date,developer,achievements,positive_ratings,negative_ratings,avg_hours_per_user,price,sells,action,adventure,rpg,simulation,strategy,racing,free_to_play,early_access
16238,Slice&Dice,2017-08-04,VRotein,17,1,0,0,3.99€,10000.0,False,False,False,True,False,False,False,False
27075,OK K.O.! Let’s Play Heroes,2018-01-22,Capybara Games,24,59,1,0,15.49€,10000.0,True,True,False,False,False,False,False,False
27076,Temple of the Apsara,2016-09-06,Attraction Studios,0,37,11,0,1.59€,10000.0,False,True,False,False,False,False,False,False
27077,The Outpost Nine: Episode 1,2018-07-27,Benjy Bates,0,22,1,0,3.99€,10000.0,False,True,False,False,False,False,False,False
27078,Ampu-Tea,2014-05-30,ProjectorGames,11,1285,754,202,3.99€,350000.0,False,True,True,True,False,False,False,False


In [190]:
# Duplicates seem to be the same game released twice on different dates, probably a marketing strategy. For example:
df[df['name'] == 'Slice&Dice']

,name,release_date,developer,achievements,positive_ratings,negative_ratings,avg_hours_per_user,price,sells,action,adventure,rpg,simulation,strategy,racing,free_to_play,early_access
14679,Slice&Dice,2017-05-31,VRotein,17,5,1,0,6.99€,10000.0,False,False,False,False,False,False,False,False
16238,Slice&Dice,2017-08-04,VRotein,17,1,0,0,3.99€,10000.0,False,False,False,True,False,False,False,False


## Fix Structural Errors

### Define Errors

- Fix the price column
- Group duplicates by name and developer
- Convert release_date to datetime format

### Fix the price column

In [191]:
df['price'] = df['price'].replace({'€': '', ',': '.'}, regex=True).astype(float)
df['price'].sample()

4365    11.39
Name: price, dtype: float64

### Group duplicates by name and developer

Games with the same name and the same developer will be grouped as follows:

- The oldest release_date will be kept.
- positive_ratings and negative_ratings will be summed.
- The average price will be calculated.
- Total sales will be summed.
- Any category with False values in duplicates will be set to True if at least one of them is True.

In [192]:
# Grouping the categories for easier handling
df_categorias = df[['action','adventure', 'rpg', 'simulation', 'strategy', 'racing']]

In [193]:
df = df.groupby(['name','developer'], as_index=False).agg({
        'release_date':'min',
        'positive_ratings':'sum',
        'negative_ratings':'sum',
        'price':'mean',
        **{categoria:'max' for  categoria in df_categorias},
        'free_to_play':'max',
        'early_access':'max',
        'achievements': 'max',
        'avg_hours_per_user': 'sum',
        'sells':'sum',
            
})

In [194]:
df[['name', 'developer']].duplicated().value_counts()

False    27073
Name: count, dtype: int64

### Convert release_date to datetime format

In [195]:
df['release_date'] = pd.to_datetime(df['release_date'], errors='coerce')
df.sample()

,name,developer,release_date,positive_ratings,negative_ratings,price,action,adventure,rpg,simulation,strategy,racing,free_to_play,early_access,achievements,avg_hours_per_user,sells
10640,Hitman: Blood Money,Io-Interactive A/S,2007-03-15,5864,391,9.29,True,False,False,False,False,False,False,False,0,435,1500000.0


In [196]:
df['release_date'] = df['release_date'].dt.strftime('%d/%m/%Y')
df.sample()

,name,developer,release_date,positive_ratings,negative_ratings,price,action,adventure,rpg,simulation,strategy,racing,free_to_play,early_access,achievements,avg_hours_per_user,sells
20114,Soldiers of the Universe,Rocwise Entertainment,30/11/2017,206,191,3.99,True,True,False,False,False,False,False,False,0,0,35000.0


In [197]:
df['release_date'] = pd.to_datetime(df['release_date'], dayfirst=True, errors='coerce')

## Filter outliers 

In [198]:
df.describe()

,release_date,positive_ratings,negative_ratings,price,achievements,avg_hours_per_user,sells
count,27073,2.707300e+04,27073.000000,27073.000000,27073.000000,27073.000000,2.707300e+04
mean,2016-12-31 13:37:09.040003072,1.000745e+03,211.096147,6.078292,45.251579,149.875337,1.341323e+05
min,1997-06-30 00:00:00,0.000000e+00,0.000000,0.000000,0.000000,0.000000,1.000000e+04
25%,2016-04-04 00:00:00,6.000000e+00,2.000000,1.690000,0.000000,0.000000,1.000000e+04
50%,2017-08-08 00:00:00,2.400000e+01,9.000000,3.990000,7.000000,0.000000,1.000000e+04
75%,2018-06-06 00:00:00,1.260000e+02,42.000000,7.190000,23.000000,0.000000,3.500000e+04
max,2019-05-01 00:00:00,2.644404e+06,487076.000000,421.990000,9821.000000,190625.000000,1.500000e+08
std,NaN,1.898942e+04,4285.102808,7.875192,352.683159,1827.113192,1.328140e+06


In [199]:
# check outliers in ratings
Q1 = df[['positive_ratings', 'negative_ratings']].quantile(0.25)
Q3 = df[['positive_ratings', 'negative_ratings']].quantile(0.75)
IQR = Q3 - Q1

# Define outlier bounds
lower = Q1 - 1.5 * IQR
upper = Q3 + 1.5 * IQR

# Filter outliers
outliers = df[((df[['positive_ratings', 'negative_ratings']] < lower) | 
               (df[['positive_ratings', 'negative_ratings']] > upper)).any(axis=1)]

outliers.sort_values(by='positive_ratings', ascending=False).head(10)

,name,developer,release_date,positive_ratings,negative_ratings,price,action,adventure,rpg,simulation,strategy,racing,free_to_play,early_access,achievements,avg_hours_per_user,sells
4813,Counter-Strike: Global Offensive,Valve;Hidden Path Entertainment,2012-08-21,2644404,402313,0.00,True,False,False,False,False,False,True,False,167,22494,75000000.0
6595,Dota 2,Valve,2013-07-09,863507,142079,0.00,True,False,False,False,True,False,True,False,0,23944,150000000.0
21937,Team Fortress 2,Valve,2007-10-10,515879,34036,0.00,True,False,False,False,False,False,True,False,520,8495,35000000.0
15999,PLAYERUNKNOWN'S BATTLEGROUNDS,PUBG Corporation,2017-12-21,496184,487076,26.99,True,True,False,False,False,False,False,False,37,22938,75000000.0
9188,Garry's Mod,Facepunch Studios,2006-11-29,363721,16433,6.99,False,False,False,True,False,False,False,False,29,12422,15000000.0
9631,Grand Theft Auto V,Rockstar North,2015-04-13,329061,139308,24.99,True,True,False,False,False,False,False,False,77,9837,15000000.0
15971,PAYDAY 2,OVERKILL - a Starbreeze Studio.,2013-08-13,308657,56523,7.49,True,False,True,False,False,False,False,False,1130,3975,15000000.0
24593,Unturned,Smartly Dressed Games,2017-07-07,292574,31482,0.00,True,True,False,False,False,False,True,False,46,3248,35000000.0
22033,Terraria,Re-Logic,2011-05-16,255600,7797,6.99,True,True,True,False,False,False,False,False,88,5585,7500000.0
12542,Left 4 Dead 2,Valve,2009-11-19,251789,8418,7.19,True,False,False,False,False,False,False,False,70,1615,15000000.0


In the case of the ratings, these values might actually be true. For example, extremely high ratings could be due to very popular games such as Dota 2 or Counter Strike.

In [200]:
# Sort the DataFrame to see the 20 most expensive games 
df.sort_values(by='price', ascending=False)[['name','price']].head(20)

,name,price
463,ADR-Labelling Game,421.99
9148,GameMaker Studio 2 UWP,303.99
10796,Houdini Indie,209.99
4981,CrisisActionVR,154.99
182,3DF Zephyr Lite Steam Edition,154.99
21768,Tactics 2: War,154.99
21771,Tactics: Bludgeons Blessing,154.99
2573,Bible Test,154.99
9147,GameMaker Studio 2 Mobile,154.99
19647,Silhouette,154.99


### Observations for the Top 20 Most Expensive Games
It can be seen that most of the first 20 results are software that could justify their high price, except for a few games that, after searching them on Google, have a different price:

**Software:**
- GameMaker Studio 2 UWP
- Houdini Indie
- 3DF Zephyr Lite Steam Edition
- GameMaker Studio 2 Mobile
- Silhouette
- GameMaker Studio 2 Web
- The Music Room
- 3DCoat 4.8
- CyberLink PowerDVD 18 Ultra - Media player, etc.
- Leadwerks Game Engine

**Games with price adjustments:**
- Tactics 2: War -> 16.79€
- Tactics: Bludgeon’s Blessing -> 16.79€
- CrisisActionVR -> 19.99€

**To be removed:**
- ADR-Labelling Game -> Currently not on Steam. According to comments, it really cost that, so I will delete it.
- Bible Test -> It's insane. It really costs that, so I will remove it.

In [201]:
# Since they are irrelevant data, I will remove the first 15 results of the "most expensive games":
df.drop(df.sort_values(by='price', ascending=False).head(15).index, inplace=True)

### Convert avg_hours_per_user to hours

In [202]:
df['avg_hours_per_user'] = df['avg_hours_per_user'] / 60

In [203]:
df.sort_values(by='avg_hours_per_user', ascending=False)[['name', 'avg_hours_per_user']].head(20)


,name,avg_hours_per_user
22100,The Abbey of Crime Extensum,3177.083333
22181,The Banner Saga: Factions,1587.416667
23139,The Secret of Tremendous Corporation,1587.366667
16017,PRICE,1058.016667
3131,Boundless,920.066667
19566,Shroud of the Avatar: Forsaken Virtues,910.300000
26001,X-Plane 11,736.150000
26888,懒人修仙传,727.200000
8096,Fantasy Grounds,717.900000
19138,Screeps,646.750000


After reviewing these games on [www.howlongtobeat.com](https://www.howlongtobeat.com), it's clear that the reported playtime values are unrealistic. Therefore, I will set them to 0.

Additionally, after doing some research, I found that **Dota 2** is the most played game on Steam. So, I will set the top 16 entries to 0 as well.

In [204]:
df_sorted = df.sort_values(by='avg_hours_per_user', ascending=False)
df_sorted.loc[df_sorted.index[:16], 'avg_hours_per_user'] = 0
df = df_sorted.sort_index()

## Dealing with NANs

In [205]:
df.isna().any()

name                  False
developer             False
release_date          False
positive_ratings      False
negative_ratings      False
price                 False
action                False
adventure             False
rpg                   False
simulation            False
strategy              False
racing                False
free_to_play          False
early_access          False
achievements          False
avg_hours_per_user    False
sells                 False
dtype: bool

There are no NaN values in the dataset 🎉

## Validate our data

In [206]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 27058 entries, 0 to 27072
Data columns (total 17 columns):
 #   Column              Non-Null Count  Dtype         
---  ------              --------------  -----         
 0   name                27058 non-null  object        
 1   developer           27058 non-null  object        
 2   release_date        27058 non-null  datetime64[ns]
 3   positive_ratings    27058 non-null  int64         
 4   negative_ratings    27058 non-null  int64         
 5   price               27058 non-null  float64       
 6   action              27058 non-null  bool          
 7   adventure           27058 non-null  bool          
 8   rpg                 27058 non-null  bool          
 9   simulation          27058 non-null  bool          
 10  strategy            27058 non-null  bool          
 11  racing              27058 non-null  bool          
 12  free_to_play        27058 non-null  bool          
 13  early_access        27058 non-null  bool          


In [207]:
df.head()

,name,developer,release_date,positive_ratings,negative_ratings,price,action,adventure,rpg,simulation,strategy,racing,free_to_play,early_access,achievements,avg_hours_per_user,sells
0,Fieldrunners 2,Subatomic Studios LLC,2013-01-10,268,53,6.99,True,False,False,False,True,False,False,False,59,0.00,150000.0
1,! That Bastard Is Trying To Steal Our Gold !,WTFOMGames,2016-03-03,17,46,2.89,True,True,False,False,False,False,False,False,0,6.60,150000.0
2,!AnyWay!,GQG DEVELOPER;SGS,2018-06-06,46,22,1.69,False,True,False,False,False,False,False,False,4997,3.55,75000.0
3,!LABrpgUP!,UPandQ,2018-06-13,11,8,0.79,False,True,True,False,False,False,False,False,2021,0.00,10000.0
4,"""BUTTS: The VR Experience""",Tyler Hurd,2016-02-03,45,13,0.79,False,False,False,False,False,False,False,False,0,0.00,10000.0


## Exporting the cleaned dataset to CSV for Looker Studio

In [208]:
df.to_csv("../data/steam_games_cleaned.csv", index=False, float_format="%.2f")

# EDA

## 1. What are the top 10 most-played games?

In [209]:
# Display the Top 10 Games by Average Playtime
df_top10 = df.sort_values(by='avg_hours_per_user', ascending=False).head(10)

# Create a Bar Chart
fig = px.bar(df_top10, x='name', y='avg_hours_per_user', 
             labels={'name':'Videogames','avg_hours_per_user': 'Average playtime by user'})

fig.update_layout(title='Top 10 Most Played Games by Average Hours Played',
                  font=dict(family='Helvetica',
                            size=14,
                            color='rgb(169, 169, 169)'),
                  template="plotly_dark",
                  height=600,
                  width=1300,
                  yaxis=dict(
                      ticksuffix='h'
                  ))



fig.update_traces(
    hovertemplate='Videogame: %{x}<br>Average hours by player: %{y:.1f}h'
)

# Dark/Light Mode Toggle
fig.update_layout(
    updatemenus=[{
        'buttons': [
            {
                'label': 'Light Mode',
                'method': 'relayout',
                'args': ['template', 'plotly_white'],  
            },
            {
                'label': 'Dark Mode',
                'method': 'relayout',
                'args': ['template', 'plotly_dark'],  
            }
        ],
        'direction': 'down',  
        'showactive': True,
        'x': 1,  
        'xanchor': 'right',  
        'y': 1.2,  
        'yanchor': 'top',  
        'pad': {'r': 5, 't': 10}, 
    }]
)

fig.show()

## 2. Which is the best-selling video game genre?

Since there is no sales column, we will focus on games that are not free to play (paid), based on their positive ratings summed with the negative ones, and filter by genre. I include the negative ratings because they also come from users who purchased the game.

In [210]:
# Filter out games that are not free-to-play
df_f2p_false = df[df['free_to_play'] == False]

# Define the genres in a variable
genres = ['action', 'adventure', 'rpg', 'simulation', 'strategy', 'racing']

# Extract relevant columns: game name, positive ratings, negative ratings, and genres
df_f2p_false_genre = df_f2p_false[['name', 'sells'] + genres]
df_f2p_false_genre = df_f2p_false_genre.set_index('name')

# Sum positive and negative ratings by genre
genre_sales = df_f2p_false_genre[genres].multiply(df_f2p_false_genre['sells'], axis=0).sum()

# Create the pie chart
fig = px.pie(
    names=genre_sales.index, 
    values=genre_sales.values, 
    title="Games Sold by Genre",
    labels={'names': 'Genre', 'values': 'Sales'}
)

fig.update_layout(
    font=dict(family='Helvetica', size=12, color='gray'),
    template="plotly_dark",
    height = 600,
    width=600
)

fig.update_traces(
    hovertemplate='Genre: %{label}<br>Units Sold: %{value:,.0f}'
)

# Dark/Light Mode Button
fig.update_layout(
    updatemenus=[{
        'buttons': [
            {
                'label': 'Light Mode',
                'method': 'relayout',
                'args': ['template', 'plotly_white'],  
            },
            {
                'label': 'Dark Mode',
                'method': 'relayout',
                'args': ['template', 'plotly_dark'],  
            }
        ],
        'direction': 'up',  
        'showactive': True,
        'x': 1.2,   
        'xanchor': 'right',  
        'y': -0.1,  
        'yanchor': 'bottom',  
        'pad': {'r': 5, 't': 10}, 
    }]
)

fig.show()

## 3. Which genres have the highest average reviews?

Nos basaremos en la media de positive_ratings segun género, esta vez incluyendo los juegos f2p

In [211]:
# Create a df with only the columns we care about
df_genres = df[['positive_ratings'] + genres]

# Calculate the weighted average
genre_avg_reviews = (
    df_genres[genres].multiply(df_genres['positive_ratings'], axis=0).sum() /  
    len(df_genres[genres])  
).reset_index()
genre_avg_reviews.columns = ['genre', 'avg_positive_ratings']
genre_avg_reviews['genre'] = genre_avg_reviews['genre'].str.capitalize()

# Create a bar chart
fig = px.bar(
    genre_avg_reviews, 
    x='genre', 
    y='avg_positive_ratings', 
    labels={'genre': 'Genre', 'avg_positive_ratings': 'Average Reviews'}
)

fig.update_layout(
    title='Best Average Reviews by Genre',
    font=dict(family='Helvetica', size=14, color='rgb(169, 169, 169)'),
    template="plotly_dark",
    height=600,
    width=1300
)

fig.update_traces(
    hovertemplate='Genre: %{x}<br>Average Positive Reviews: %{y:.0f}')

# Dark/Light Mode Button
fig.update_layout(
    updatemenus=[{
        'buttons': [
            {
                'label': 'Light Mode',
                'method': 'relayout',
                'args': ['template', 'plotly_white'],  
            },
            {
                'label': 'Dark Mode',
                'method': 'relayout',
                'args': ['template', 'plotly_dark'],  
            }
        ],
        'direction': 'down',  
        'showactive': True,
        'x': 1,  
        'xanchor': 'right',  
        'y': 1.2,  
        'yanchor': 'top',  
        'pad': {'r': 5, 't': 10}, 
    }]
)
fig.show()

## 4. How much has the top-selling developer earned?

In [212]:
df_devs = df[['developer','sells', 'price']].copy()
df_devs['earnings'] = df['sells'] * df['price']
developer_earnings = df_devs.groupby('developer')['earnings'].sum()
top_10_developers = developer_earnings.sort_values(ascending=False).head(5)

# Create a bar chart
fig = px.bar(
    top_10_developers, 
    x=top_10_developers.index, 
    y=top_10_developers.values,
    title="Top 5 Developers who earned the most € according to their sales",
    labels={'x': 'Developer', 'y': 'Total Earnings'},
    template="plotly_dark"
)

fig.update_layout(
    font=dict(family='Helvetica', size=14, color='rgb(169, 169, 169)'),
    template="plotly_dark",
    height=600,
    width=1300,
    xaxis_title='Developer',
    yaxis=dict(
        ticksuffix='€',
    )
)

fig.update_traces(
    hovertemplate='Developer: %{x}<br>Earnings: %{y:,.0f}€'
)

# Light/Dark Mode Button
fig.update_layout(
    updatemenus=[{
        'buttons': [
            {
                'label': 'Light Mode',
                'method': 'relayout',
                'args': ['template', 'plotly_white'],  
            },
            {
                'label': 'Dark Mode',
                'method': 'relayout',
                'args': ['template', 'plotly_dark'],  
            }
        ],
        'direction': 'down',  
        'showactive': True,
        'x': 1,  
        'xanchor': 'right',  
        'y': 1.2,  
        'yanchor': 'top',  
        'pad': {'r': 5, 't': 10},  
    }]
)

fig.show()

## 5. How do Early Access games affect review scores?

In [213]:
# Define variables
df_early_access = df[df['early_access'] == True]
avg_positive_reviews_early_access = df_early_access['positive_ratings'].mean()
avg_negative_reviews_early_access = df_early_access['negative_ratings'].mean()

# Prepare data for the pie chart
review_data = {
    'Review Type': ['Positive Reviews', 'Negative Reviews'],
    'Average Reviews': [avg_positive_reviews_early_access, avg_negative_reviews_early_access]
}

# Create the pie chart
fig = px.pie(
    review_data, 
    names='Review Type', 
    values='Average Reviews', 
    title="Average Reviews for Early Access Games",
    color='Review Type', 
    color_discrete_map={'Positive Reviews': 'green', 'Negative Reviews': 'red'}
)

# Update layout for dark mode and other settings
fig.update_layout(
    font=dict(family='Helvetica', size=14, color='rgb(169, 169, 169)'),
    template="plotly_dark",  
    height=600,
    width=800
)

# Light/Dark Mode Button
fig.update_layout(
    updatemenus=[{
        'buttons': [
            {
                'label': 'Light Mode',
                'method': 'relayout',
                'args': ['template', 'plotly_white'],  
            },
            {
                'label': 'Dark Mode',
                'method': 'relayout',
                'args': ['template', 'plotly_dark'],  
            }
        ],
        'direction': 'down',  
        'showactive': True,
        'x': 1,  
        'xanchor': 'right',  
        'y': 1.2,  
        'yanchor': 'top',  
        'pad': {'r': 5, 't': 10},  
    }]
)

fig.show()

## 6. Which developer(s) have released the most diverse and successful games?

In [214]:
df_devs_genres = df[['developer'] + genres].copy()

# Convert the genre columns to long format
df_devs_genres_long = df_devs_genres.melt(id_vars='developer', value_vars=genres, var_name='genre', value_name='has_genre')

# Filter only the games that have a genre (if 'has_genre' is 1)
df_devs_genres_long = df_devs_genres_long[df_devs_genres_long['has_genre'] == 1]

# Count the number of games per genre for each developer
developer_genre_count = df_devs_genres_long.groupby(['developer', 'genre']).size().reset_index(name='count')

# Select the top 3 developers with the most game types
top_3_developers = developer_genre_count.groupby('developer')['count'].sum().sort_values(ascending=False).head(3)

# Filter for the top 3 developers
top_3_devs_data = developer_genre_count[developer_genre_count['developer'].isin(top_3_developers.index)]

# Create the stacked bar chart
fig = px.bar(
    top_3_devs_data,
    x='developer',
    y='count',
    color='genre',
    title="Top 3 Developers with the Most Game Types Developed and Their Genre Distribution",
    labels={'x': 'Developer', 'y': 'Number of Games per Genre'},
    barmode='stack'  
)

fig.update_layout(
    font=dict(family='Helvetica', size=14, color='rgb(169, 169, 169)'),
    template="plotly_dark",
    height=600,
    width=1300
)

# Light/Dark Mode Button
fig.update_layout(
    updatemenus=[{
        'buttons': [
            {
                'label': 'Light Mode',
                'method': 'relayout',
                'args': ['template', 'plotly_white'],  
            },
            {
                'label': 'Dark Mode',
                'method': 'relayout',
                'args': ['template', 'plotly_dark'],  
            }
        ],
        'direction': 'down',  
        'showactive': True,
        'x': 1,  
        'xanchor': 'right',  
        'y': 1.2,  
        'yanchor': 'top',  
        'pad': {'r': 5, 't': 10},  
    }]
)
fig.show()

## 7. Which month saw the highest number of game releases?

In [215]:
# Create a new column that only shows the month and year of release
df['release_month'] = df['release_date'].dt.to_period('M')

# Count how many games were released each month
games_per_month = df['release_month'].value_counts().sort_index()

# Find the highest point
max_point = games_per_month.idxmax()
max_value = games_per_month.max()
max_point_formatted = max_point.strftime('%m/%Y')


# Create a line chart
fig = px.line(
    games_per_month,
    x=games_per_month.index.astype(str),
    y=games_per_month.values,
    title="Games Released per Month from 1997 to 2019",
    labels={'x': 'Release Month', 'y': 'Number of Games Released'},
    template="plotly_dark",
)

# Add the highest point marker
fig.add_scatter(
    x=[str(max_point)],  
    y=[max_value],
    mode='markers+text',
    marker=dict(color='red', size=10, symbol='star'),
    text=[f'{max_point_formatted} - {max_value} Games'],
    textposition='top left',
    showlegend=False
)


fig.update_layout(
    font=dict(family='Helvetica', size=14, color='rgb(169, 169, 169)'),
    height=600,
    width=1000
)

# Light/Dark mode button
fig.update_layout(
    updatemenus=[{
        'buttons': [
            {
                'label': 'Light Mode',
                'method': 'relayout',
                'args': ['template', 'plotly_white'],  
            },
            {
                'label': 'Dark Mode',
                'method': 'relayout',
                'args': ['template', 'plotly_dark'],  
            }
        ],
        'direction': 'down',  
        'showactive': True,
        'x': 1.1,  
        'xanchor': 'right',  
        'y': 1.2,  
        'yanchor': 'top',  
        'pad': {'r': 5, 't': 10},  
    }]
)

fig.show()

## 8. How many hours are played per genre on average each month?

In [216]:
categories = ['action', 'adventure', 'rpg', 'simulation', 'strategy', 'racing']

# Create new columns for hours played by genre
for genre in categories:
    df[f'{genre}_hours'] = df['avg_hours_per_user'] * df[genre]
    
# Group by month and sum the total hours
genre_hours_monthly = df.groupby('release_month')[
    [f'{genre}_hours' for genre in categories]
].sum()

# Reset the index so 'release_month' becomes a column
genre_hours_monthly = genre_hours_monthly.reset_index()

# Convert 'release_month' to string to ensure compatibility with Plotly
genre_hours_monthly['release_month'] = genre_hours_monthly['release_month'].astype(str)

# Convert the DataFrame to long format for Plotly
genre_hours_long = genre_hours_monthly.melt(
    id_vars='release_month',
    var_name='genre',
    value_name='hours'
)
genre_hours_long['genre'] = genre_hours_long['genre'].replace({
    'action_hours': 'Action',
    'adventure_hours': 'Adventure',
    'rpg_hours': 'RPG',
    'simulation_hours': 'Simulation',
    'strategy_hours': 'Strategy',
    'racing_hours': 'Racing'
})

In [217]:
# Crear el gràfic d'àrea apilada
fig = px.area(
    genre_hours_long,
    x='release_month',
    y='hours',
    color='genre',
    title='Evolución de las horas jugadas por mes por género',
    labels={'release_month': 'Timeline', 'hours': 'Horas jugadas', 'genre': 'Genero'},
    template='plotly_dark'
)

fig.update_layout(
    font=dict(family='Helvetica', size=14, color='rgb(169, 169, 169)'),
    height=600,
    width=1300,
    legend_title="Género",
    yaxis=dict(
        ticksuffix='h'
    )
)

fig.update_traces(
    hovertemplate='%{x}<br>Horas jugadas: %{y:.1f}h'
)

# Botón Claro/Oscuro
fig.update_layout(
    updatemenus=[{
        'buttons': [
            {
                'label': 'Modo Claro',
                'method': 'relayout',
                'args': ['template', 'plotly_white'],  
            },
            {
                'label': 'Modo Oscuro',
                'method': 'relayout',
                'args': ['template', 'plotly_dark'],  
            }
        ],
        'direction': 'down',  
        'showactive': True,
        'x': 1.1,  
        'xanchor': 'right',  
        'y': 1.2,  
        'yanchor': 'top',  
        'pad': {'r': 5, 't': 10},  
    }]
)

fig.show()

## 9. Do Free-to-Play games have better or worse reviews?

In [218]:
# Filter data and define functions
df_f2p = df[df['free_to_play'] == True]
df_f2p_positive = df_f2p['positive_ratings'].sum()
df_f2p_negative = df_f2p['negative_ratings'].sum()

# Create values to display on the pie chart
labels = ['Positive Ratings', 'Negative Ratings']
values = [df_f2p_positive, df_f2p_negative]

# Create the pie chart
fig = px.pie(
    names=labels,
    values=values,
    title="Distribution of Ratings in Free-to-Play Games",
    template="plotly_dark"
)

fig.update_layout(
    font=dict(family='Helvetica', size=12, color='gray'),
    template="plotly_dark",
    height = 600,
    width=600
)

fig.update_traces(
    hovertemplate='%{label}: %{value:,.0f}'
)

# Light/Dark Mode Button
fig.update_layout(
    updatemenus=[{
        'buttons': [
            {
                'label': 'Light Mode',
                'method': 'relayout',
                'args': ['template', 'plotly_white'],  
            },
            {
                'label': 'Dark Mode',
                'method': 'relayout',
                'args': ['template', 'plotly_dark'],  
            }
        ],
        'direction': 'up',  
        'showactive': True,
        'x': 1.2,   
        'xanchor': 'right',  
        'y': -0.1,  
        'yanchor': 'bottom',  
        'pad': {'r': 5, 't': 10}, 
    }]
)
fig.show()

## 10. How has the number of achievements per game evolved over time?

In [219]:
# Create the scatter plot
fig = px.scatter(
    df,
    x='release_date',
    y='achievements',
    title='Evolution of Achievements per Game',
    labels={'release_date': 'Date', 'achievements': 'Achievements', 'name': 'Game Name'},
    template='plotly_dark'
)

fig.update_traces(textposition='top center')

fig.update_layout(
    font=dict(family='Helvetica', size=14, color='rgb(169, 169, 169)'),
    height=600,
    width=1300
)

fig.update_traces(
    customdata=df['name'],
    hovertemplate='%{x}<br>Game: %{customdata}<br>Achievements: %{y}'
)

# Light/Dark Mode Button
fig.update_layout(
    updatemenus=[{
        'buttons': [
            {
                'label': 'Light Mode',
                'method': 'relayout',
                'args': ['template', 'plotly_white'],  
            },
            {
                'label': 'Dark Mode',
                'method': 'relayout',
                'args': ['template', 'plotly_dark'],  
            }
        ],
        'direction': 'down',  
        'showactive': True,
        'x': 1.1,  
        'xanchor': 'right',  
        'y': 1.2,  
        'yanchor': 'top',  
        'pad': {'r': 5, 't': 10},  
    }]
)

fig.show()

The team at Ripknot Studio are the ones who add 5,000 achievements to all their games.